# Исследование надежности заемщиков
---

## Контекст задачи

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

## Инструкция по выполнению

`Шаг 1.` Откроем таблицу и изучим общую информацию о данных


`Шаг 2.` Предобработка данных

* Обработаем пропущенные данные 
* Найдем артефакты(аномалии)
* Заменим вещественные типы данных
* Обработаем дубликаты
* Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`

`Шаг 3.` Ответим на вопросы
 
* Узнаем, есть ли зависимость между количеством детей и возвратом кредита в срок
* Узнаем, есть ли зависимость между семейным положением и возвратом кредита в срок
* Узнаем, есть ли зависимость между уровнем дохода и возвратом кредита в срок
* Узнаем, как разные цели кредита влияют на его возврат в срок

`Шаг 4.` Напишем общий вывод



## Описание данных

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

##   Обзор данных

In [2]:
import pandas as pd

In [3]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('/Users/ivan_miroshnichenko/Downloads/datasets/data.csv')

In [ ]:
display(data.tail(10))
data.info()

##  Заполнение пропусков

Обнаружены пропуски в столбцах `days_employed` и `total_income`. Скорее всего пропуски в этих столбцах связаны с тем, что пользователь решил не заполнять их в форме заявки банка. Количество пропусков в обоих столбцах одинаковое. `Пропуски` составляют `10%` от обшего числа данных. Это весомое, но не очень критичное количество. Для дальнейшей корректной работы эти пропуски необходимо заполнить. В данном случае, так как речь идет о количественных переменных, верннее всего было бы заполнить пропуски значением медианы для каждого столбца. Выбираем медиану потому что она робастна к выбросам, то есть исключает влияние преобладания крайне высоких и крайне низких значений. 

In [4]:
display(data.isna().sum())
#print(data['total_income'].isna().sum() / (len(data['total_income']) / 100)) #какой процент от общего числа значений составляют пропуски
display(data['total_income'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.info()



children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

145017.93753253992

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбцах `days_employed` и `children` найдены аномалии с отрицательными значениями в столбцах. В случае с `days_employed` возможно это связано с некорректным переводом в `unix time` формат. Чтобы решить эту проблему, привели все значения по модулю.

In [5]:
display(f"Максимальный трудовой стаж: {data['days_employed'].max()}; Минимальный трудовой стаж: {data['days_employed'].min()}")
display(f"Максимальное кол-во детей: {data['children'].max()}; Минимальное кол-во детей: {data['children'].min()}")
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data.info()

'Максимальный трудовой стаж: 401755.40047533; Минимальный трудовой стаж: -18388.949900568383'

'Максимальное кол-во детей: 20; Минимальное кол-во детей: -1'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Проверка данных на аномалии и исправления.

In [6]:
data.info()
display(data['children'].value_counts())
data.loc[data['children'] == 20, 'children'] = 2
#display(data.loc[data['children'] == 20])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

## Изменение типов данных.

In [7]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

## Удаление дубликатов.

В столбце `education` встречаются дубликаты значений записаные разным регистром. Наиболее быстрый способ избавиться от дубликатов - применить метод `lower()` к каждому значению столбца. Дубликаты появились из-за того что пользователь не выбирал статус образования заданый формой, а заполнял форму сам, поэтому множились различные варианты записи одних и тех же статусов.

In [8]:
#print(f"Общее кол-во дубликатов: {data.duplicated().sum()}")
data = data.drop_duplicates().reset_index(drop=True)
#print(data['education'].value_counts())

data['education'] = data['education'].str.lower()

data['education'].value_counts() #проверка замены дубликатов

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [9]:
education_dict = data[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict.sort_values('education_id'))

family_status_dict = data[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.sort_values('family_status_id'))


   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень
   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


In [10]:
data.pop('education')
data.pop('family_status')
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21467,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


## Категоризация дохода.

In [11]:
def total_income_group(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000: 
        return 'D'
    if 50001 <= total_income <= 200000: 
        return 'C'
    if 200001 <= total_income <= 1000000: 
        return 'B' 
    if total_income > 1000001:
        return 'A'
    

In [12]:
data['total_income_category'] = data['total_income'].apply(total_income_group)    
data['total_income_category'].value_counts()

C    16033
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

## Категоризация целей кредита.

In [13]:
def purpose_group(purpose):
    try:
        if 'образов' in purpose:
            return 'получение образования'
        if 'авто' in purpose:
            return 'операции с автомобилем' 
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'жиль' or 'недвиж' in purpose:
            return 'операции с недвижимостью'
    except:
        return 'неизвестная категория'

In [14]:
data['purpose_category'] = data['purpose'].apply(purpose_group)
data['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

## Ответы на вопросы.

### Зависимость между количеством детей и возвратом кредита в срок.
Чтобы ответить на вопрос есть ли зависимость между количеством детей и возвратом кредита в срок, нам необходимо создать сводную таблицу по количеству детей и наличию или отсутствию долга.

In [15]:
debt_to_child_pivot = data.pivot_table(index='children', values='debt', aggfunc=sum)
debt_to_child_pivot['without_debt'] = data['children'].value_counts()
debt_to_child_pivot['share_of_debt(%)'] = debt_to_child_pivot['debt'] / (debt_to_child_pivot['without_debt'] /100)
debt_to_child_pivot

,debt,without_debt,share_of_debt(%)
children,,,
0,1063,14107,7.535266
1,445,4856,9.163921
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


#### Вывод:
По сводной таблице выше видно, что кол-во детей не влияет на возвращаемость кредита, потому что в каждой из категорий процент клиентов с долгом 7-9%. Однако, наименьший процент все таки составляет категория заемщиков без детей.

### Зависимость между семейным положением и возвратом кредита в срок.
Чтобы ответить на вопрос есть ли Зависимость между семейным положением и возвратом кредита в срок, нам необходимо создать сводную таблицу по количеству детей и наличию или отсутствию долга.

In [16]:
debt_to_family_status_pivot = data.pivot_table(index='family_status_id', values='debt', aggfunc=sum)
debt_to_family_status_pivot['without_debt'] = data['family_status_id'].value_counts()
debt_to_family_status_pivot['share_of_debt(%)'] = debt_to_family_status_pivot['debt'] / (debt_to_family_status_pivot['without_debt'] /100)
debt_to_family_status_pivot 

,debt,without_debt,share_of_debt(%)
family_status_id,,,
0,931,12344,7.542126
1,388,4163,9.320202
2,63,959,6.569343
3,85,1195,7.112971
4,274,2810,9.750890


#### Вывод:
По сводной таблице выше видно, что вдовы/вдовцы, разведенные и люди состоящие в браке реже имеют задолженности по кредиту, чем люди состоящие в  гражданском браке или не замужем/не женатые.

### Зависимость между уровнем дохода и возвратом кредита в срок.

In [17]:
debt_to_income_pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=sum)
debt_to_income_pivot['without_debt'] = data['total_income_category'].value_counts()
debt_to_income_pivot['share_of_debt(%)'] = debt_to_income_pivot['debt'] / (debt_to_income_pivot['without_debt'] /100)
debt_to_income_pivot

,debt,without_debt,share_of_debt(%)
total_income_category,,,
A,2,25,8.000000
B,356,5041,7.062091
C,1360,16033,8.482505
D,21,350,6.000000
E,2,22,9.090909


#### Вывод:
По сводной таблице выше видно, что люди с доходом выше среднего(B) и с доходом ниже среднего(D) чаще возвращают кредиты.

### Как разные цели кредита влияют на его возврат в срок?

In [18]:
debt_to_purpose_pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc='sum')
debt_to_purpose_pivot['without_debt'] = data['purpose_category'].value_counts()
debt_to_purpose_pivot['share_of_debt(%)'] = debt_to_purpose_pivot['debt'] / (debt_to_purpose_pivot['without_debt'] /100)
debt_to_purpose_pivot 

,debt,without_debt,share_of_debt(%)
purpose_category,,,
операции с автомобилем,403,4308,9.354689
операции с недвижимостью,782,10814,7.231367
получение образования,370,4014,9.217738
проведение свадьбы,186,2335,7.965739


#### Вывод:
По сводной таблице выше видно, что наибольшие риски с точки зрения задолженности несут кредиты выданные на операции с автомобилем и образование, в то время как операции с недвижимостью имеют меньший риск.

---
## Общий вывод:

На основании исследования можно сделать вывод, что наибольшей группой риска являются клиенты, несостоящие в браке, имеющие детей, имеющие средний уровень дохода, а целью оформления кредита являются операции с автомобилем или образованием. Наиболее благоприятными замещиками являются семейные пары, неимеющие детейл, с доходом ниже или выше среднего.